# Testing For SSD



1. Get the data
2. [DOING .. ] Create the base network.
   1. [DOING .. ] Create the base network. 
   2. [DOING .. ] Base network should send out to final predctions a class predction (1-hot encoded) and a location prediction (a,b,c,d) relative sizes. 
   2. [DOING .. ] Figure out which layers will output detections to prediction array. 
   2. [DOING .. ] Figure out the size of the feature maps in each case above. 
   3. [DOING .. ] Figure out number of outputs in final array 
3. Preprocess
  1. Create a function to pull a image and its data 
  2. create a hash of Ground truth image -> bounding box, class
  3. Create the default box sizes.
  3. For each feature map square, create the different default boxes. 
     1. For each default box 
         2. See if it matches with the ground truth.
             3. YES - then set class value to 1 (pedestrian)
             4. NO - set class value to 0 (background)
         3. Calculate how much negative boxes are created. If too many ( greater than 2/3 ratio) then drop from training set. 
         4. Create a list of [y_box_coords, y_confidence (1-hot encoded class),]
4. Create one prediction through the network. 
4. Data Augmenation 
   1. Use the entire orignal input image
   2. Sample a patch so that the mininmum jaccard overlap with the object is 0.1,0.3,0.5,0.7,0.9
   3. Randomly Sample a patch. 
   4. This new image needs to be run through the same pipeline. 
5. Loss should be cross-entropy for the class predictions and mse for the coordinates. 
3. Change input size of the image to match Pedestrian dataset.
5. Train the system on pedestrian dataset. 
5. Experiment with different feature map sizes (skinny is better I think.)
5. Run predictions from test set to 
5. Experiment with different sizes to see if it varies the accuracy... 
6. Can I load the imagenet weights into the system ? 

#### Initial System 
1. Get base network running.
2. Get base loss and. 
3. No augmentation. 
4. 

## Get the data

2. Download base dataset and run it through it. 

In [ ]:
IMAGE_H = 300
IMAGE_W = 300



In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
import numpy as np

class IllegalArgumentError(ValueError):
    pass

def conv_layer_optional_pooling(x_tensor,n_outputs,n_ksize,n_strides,name,
                                padding_type="VALID",pool_ksize=None,pool_strides=None,pool_name=None):
    # Convolution layer with Relu 
    """
    x_tensor : input tensor
    n_outputs: number of outputs of the convolutional layer
    n_ksize: kernel size 2-d tuple 
    n_strides: 2-d tuple for convlution
    padding_type: Type of padding. default is "SAME"
    pool_ksize: kernel size 2-d tuple for max pool
    pool_strides: stride 2-d tuple for max pool
    
    returns A tensor that is hte output of convolution, relu & max-pooling (optional)
    """
    
    # check for dumb input errors
    if (len(n_ksize) != 2) or 
        (len(n_strides) !=2) or 
        (pool_ksize!=None and len(pool_ksize)!=2) or (pool_strides!=None and len(pool_strides!=2)):
        raise IllegalArgumentError
        
    num_channels = int(x_tensor.shape[-1])
    
    filter_weight = tf.Variable(tf.truncated_normal(list(n_ksize)+[num_channels,n_outputs],mean=0,stddev=0.1))
    filter_bias = tf.Variable(tf.zeros(n_outputs))
    
    conv_layer = tf.nn.conv2d(x_tensor,filter_weight,[1]+list(n_strides)+[1],padding_type,name=name)
    conv_layer = tf.nn.bias_add(conv_layer,filter_bias)
    conv_layer = tf.nn.relu(conv_layer)

    print("After conv_layer ",name,conv_layer.shape)
    
    if pool_ksize!= None and pool_strides !=None:
            pooled = tf.nn.max_pool(conv_layer,
                                    ksize=[1] + list(pool_ksize) + [1],
                                    strides = [1] + list(pool_strides) + [1],
                                    padding='SAME',name=pool_name)
            print("After pool ",pool_,x.shape)
            return pooled
        
    return conv_layer

def flatten(x_tensor):
    batch_size = x_tensor.shape[0]
    for a in range(1,len(x_tensor.shape)):
        mult = mult * int(x_tensor.shape[a])
    return tf.reshape(x_tensor,[-1,mult])

def fully_conn(x_tensor,num_outputs):
    num_inputs = int(x_tensor.shape[1])
    weight= tf.Variable(tf.random_normal([num_inputs,num_outputs],mean=0,stddev=0.1))
    bias = tf.Variable(tf.zeros(shape=num_outputs))
    
    layer = tf.add(tf.matmul(x_tensor,weight),bias)
    layer = tf.nn.relu(layer)
    return layer
    
    
    
def convolve_and_collect(x):
    # Apply 2 convolutions
    
def VGGModel(x):
    """
    This will have two outputs - the coordinates for all default boxes and confidences
    """
    
    x = conv_layer_optional_pooling(x,64,(3,3),(1,1),"block1_conv1",padding_type="SAME")
    x = conv_layer_optional_pooling(x,64,(3,3),(1,1),"block1_conv2",padding_type="SAME",
                                    pool_ksize=(2,2),pool_strides=(2,2),pool_name='block1_pool')
    print("x after block1_pool",x.shape)
    
    x = conv_layer_optional_pooling(x,128,(3,3),(1,1),"block2_conv1",padding_type="SAME")
    x = conv_layer_optional_pooling(x,128,(3,3),(1,1),"block2_conv2",padding_type="SAME",
                                    pool_ksize=(2,2),pool_strides=(2,2),pool_name='block2_pool')
    print("x after block2_pool",x.shape)
    
    x = conv_layer_optional_pooling(x,256,(3,3),(1,1),"block3_conv1",padding_type="SAME")
    x = conv_layer_optional_pooling(x,256,(3,3),(1,1),"block3_conv2",padding_type="SAME")
    x = conv_layer_optional_pooling(x,256,(3,3),(1,1),"block3_conv3",padding_type="SAME",
                                    pool_ksize=(2,2),pool_strides=(2,2),pool_name='block3_pool')
    print("x after block3_pool",x.shape)
    
    x = conv_layer_optional_pooling(x,512,(3,3),(1,1),"block4_conv1",padding_type="SAME")
    x = conv_layer_optional_pooling(x,512,(3,3),(1,1),"block4_conv2",padding_type="SAME")
    x = conv_layer_optional_pooling(x,512,(3,3),(1,1),"block4_conv3",padding_type="SAME",
                                    pool_ksize=(2,2),pool_strides=(2,2),pool_name='block4_pool')
    
    print("x after block4_pool",x.shape)
    # Need the final 3 layers with the image maps

    # Add slowly reducing layers after this till we get to 1 pixel size region. 
    x = flatten(x)
    
def getData():
    """
    Get the data and create a hash of all whats needed
    
    """
    for 
    
def train(x):
    
    # Get the input 
    
    x = tf.placeholder(tf.float32,shape=(None,IMG_W,IMAGE_H,name='x'))
    y = tf.placeholder(tf.float32,shape=(None,TOTAL_Y_LEN,name='y'))
    
    # Inititalize the model
    logits = VGGModel(x)
    
    # Figre out the costs. We will have a sum of two costs. 
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logist=logits,labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    
    with tf.Session as sess:
            sess.run(tf.global_variables_initializer())
            
            for epoch in range(epochs):
                n_batches = 5 
                for batch_i in range(1,n_batches+1):
                    # Get the input
                    # run the optimizer
                    session.run(optimizer,feed_dict={x:,y:})
        
                       
    

SyntaxError: invalid syntax (<ipython-input-1-c006d4cdb479>, line 23)

In [ ]:
## Test base network.

In [ ]:
## 